# Question 8
O problema apresentado envolve o uso do método da máxima verossimilhança para estimar os parâmetros de um modelo probabilístico de um comitê de redes especialistas, onde uma rede adicional indica qual das redes especialistas é mais adequada para processar uma entrada específica. A função log de verossimilhança deve ser maximizada em relação aos parâmetros w (pesos das redes especialistas) e a (pesos da rede indicadora). Para fazer isso, utilizamos o método do gradiente da descida mais íngreme.

A função likelihood, denotada por $l(\theta)$, pode ser escrita com base na função densidade de probabilidade. Como é mais conveniente trabalhar com o log-likelihood, temos:
$$l(w, g) = ln [f (d | x)]$$

Assim, a função log-likelihood é dada por:

$$l(w, a) = ln \sum_{i=1}^K g_i \exp(-\frac{1}{2}||d-y_i||^2)$$

A seguir, temos $g_i$, que corresponde à ativação da rede indicadora. Esta equação define a ativação softmax da rede indicadora $g_i(n)$. A função softmax é aplicada à ativação da rede indicadora $u_i(n)$, calculada na etapa anterior. A ativação softmax produz uma distribuição de probabilidade entre as redes especialistas, garantindo que a soma de todas as ativações softmax seja igual a 1.

$$g_i = \frac{\exp(u_i)}{\sum_{j=1}^K\exp(u_j)}$$

Depois, tem-se $h_i$, que corresponde à probabilidade de saída da rede especialista i:
$$h_i(n) = \frac{g_i(n)\exp(-\frac{1}{2}||d-y_i(n)||)}{\sum_{j=1}^K g_j(n)\exp(-\frac{1}{2}||d-y_j(n)||)}$$

onde $$\\i = 1, 2, ... , K$$

Dessa maneira, $h_i$ deve satisfazer as seguintes condições:
$$1 \leq h_i(n) \leq 1 \text{ e } \sum_{i=1}^K h_i(n) = 1$$

Logo a seguir, tem-se $e_i^m$, que corresponde ao erro da rede especialista i para a amostra m. O erro é calculado como a diferença entre o vetor de saída desejado $d^m$ e o vetor de saída da rede especialista $y_i^m(n)$:
$$e_i^m = d^m - y_i^m$$

$\frac{∂l} {∂w_{i}^m}$ corresponde ao gradiente da função log-likelihood em relação aos pesos da rede especialista i para a amostra m. Assim, temos:
$$\frac{∂l} {∂w_{i}^m} = \frac{∂l}{∂y_i^m} \frac{∂y_i^m}{∂w_i^m}$$

Já $\frac{∂l} {y_{i}^m}$ é obtido utilizando $y_i^m = f_i(x^m, w_i^m)$ e a regra da cadeia, resultando em:

$$\frac{∂l} {y_{i}^m} = h_i(d^m - y_i^m) = h_ie_i^m$$

Para obter $\frac{∂l} {w_{i}^m} = h_ie_i^mx$, é necessário calcular $\frac{∂y_i^m}{∂w_i^m}$, que é obtido utilizando a regra da cadeia e a função de ativação $f_i$, resultando em:

$$\frac{∂l} {w_{i}^m} = h_ie_i^mx$$

onde
$$i = 1, 2, ... ,K$$
$$m = 1, 2, ... ,q$$

Aplicando um pequeno ajuste na equação, temos:
$$\Delta w_i^m = η\frac{∂l} {w_{i}^m}$$

onde $η$ é um pequeno parâmetro learning rate. É possível perceber que o lado direito da equação possui $+η$, uma vez que estamos usando o gradiente ascendente para maximizar $l$. Então, usando $w_i^m(n)$ para denotar o valor do vetor de peso sináptico $w_i^m$ em cada iteração $n$, o valor atualizado do vetor de peso sináptico em cada  iteração $n+1$ é computado de acordo com a seguinte recursão:

$$(w_i^m)(n+1) = (w_i^m)(n) + \Delta w_i^m(n)$$
$$(w_i^m)(n+1) = (w_i^m)(n) + \eta h_ie_i^m(n)x$$

A equação abaixo define a ativação da rede indicadora $u_i$ antes da função softmax ser aplicada. A ativação é calculada como o produto interno entre o vetor de entrada transposto $x^T$ e o vetor de pesos da rede indicadora $a_i(n)$ na iteração $n$:
$$u_i = x^Ta_i$$
$$\frac{∂u_i} {∂a_{i}} = x$$


Agora, temos $\frac{∂l} {∂a_{i}}$, que corresponde ao gradiente da função log-likelihood em relação aos pesos da rede indicadora:
$$\frac{∂l} {∂a_{i}} = \frac{∂l}{∂u_i} \frac{∂u_i}{∂a_i}$$
$$\frac{∂l} {∂a_{i}} = (h_i - g_i)x$$

Encontrando $\frac{∂l} {∂a_{i}}$, temos:
$$\Delta{a_i} = η\frac{∂l}{∂a_i}$$
$$\Delta{a_i} = η(h_i - g_i)x$$

O valor de $a_i$ em cada iteração $n+1$ é atualizado de acordo com a seguinte recursão:

$$a_i(n+1) = a_i(n) + η[h_i(n) - g_i(n)]x$$

Assim:


$u_i = x^Ta_i(n)$

$g_i(n) = \frac{\exp(u_i(n))}{\sum_{j=1}^K\exp(u_j(n))}$

$y_i^m(n) = x^Tw_i^m(n)$

$y_i(n) = [y_i^1(n), y_i^2(n), ..., y_i^q(n)]^T$

$h_i(n) = \frac{g_i(n)\exp(-\frac{1}{2}||d-y_i(n)||)}{\sum_{j=1}^K g_j(n)\exp(-\frac{1}{2}||d-y_j(n)||)}$

$e_i^m = d^m - y_i^m(n)$

$(w_i^m)(n+1) = (w_i^m)(n) + \eta h_ie_i^m(n)x$

$a_i(n+1) = a_i(n) + η[h_i(n) - g_i(n)]x$



Baseado nas demonstrações acima, detalharemos o passo a passo do algoritmo de treinamento do comitê de redes especialistas:
1. Inicialize os pesos sinápticos $w_i^m$ e $a_i$ com valores aleatórios pequenos.
2. Para cada padrão de treinamento $x(n)$, calcule $u_i(n)$, $g_i(n)$, $y_i^m(n)$, $y_i(n)$, $h_i(n)$, $e_i^m$, $(w_i^m)(n+1)$ e $a_i(n+1)$.
3. Repita o passo 2 até que o critério de parada seja satisfeito.
4. Itere os passos 2 e 3 até que o critério de parada seja satisfeito.